In [1]:
import numpy as np
from astropy.table import Table, join, MaskedColumn, vstack
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy
from astropy.time import Time
import pandas as pd
import re
import seaborn as sns
import datetime
from datetime import datetime
from datetime import timedelta
from math import e
from math import pi
from astropy.table import Column
from math import sqrt
import numpy as np
import emcee
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
import math
import corner
from numpy import exp
from scipy import integrate
from scipy.integrate import quad
import pdb
import powerlaw
import random

In [31]:
AP_data_table=Table.read("APData_csv.csv")
print("AP_data_table:", AP_data_table.colnames)

M=np.array(AP_data_table['logMass-Best'])
Tau= np.array(AP_data_table['logAge-Best'])

def M_lim(Tau):
    if 7.0 < Tau < 8.0 :
        return np.log10(740)
    if 8.0 < Tau < np.log10(300000000) : 
        return np.log10(1080)
    else:
        return 99
    
M_lim_a=np.zeros((len(M)))
for i in range(len(M)):
    M_lim_a[i]=M_lim(Tau[i])

#Ob_Com needs to be in log space 
def lnobs_like(M, M_lim_a):
    a_lim=5.0
    for i in range(len(M)):
        if M[i] > M_lim_a[i]:
            return -np.log(1+ exp(-a_lim*(M[i]-M_lim_a[i])))
        else:
            return -np.inf
        
def lnZ(theta, M, M_lim_a):
    alpha, M_c = theta
    a_lim=5.0
    lin_M_c= 10**M_c
    def f(M, M_lim_a):
        if M > 10**M_lim_a:
            return (M**alpha)*exp(-M/lin_M_c)*((1+ exp(-a_lim*(np.log10(M)-M_lim_a))**(-1)))
        else:
            return 0
    ans=np.zeros((len(M)))
    for i in range(len(M)):
        ans[i], err = quad(f, 10**M_lim_a[i], np.inf, args=(M_lim_a[i]))
        if ans[i] ==0 :
            return -np.inf
    return np.log(ans)
    
def lnlike(theta, M, M_lim_a):
    alpha, M_c = theta
    lin_M= 10**M
    lin_M_c= 10**M_c
    return np.sum((-lin_M/lin_M_c) + alpha*np.log(lin_M) + lnobs_like(M, M_lim_a) - lnZ(theta, lin_M, M_lim_a))
    
def lnprior(theta):
    alpha, M_c = theta
    if -3 <= alpha <= -1 and 3 <= M_c <= 8:
        return 0.0
    return -np.inf

def lnprob(theta, M, M_lim_a):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, M, M_lim_a)

starting_point=np.array([-1.99, 3.93])

ndim, nwalkers = 2, 500
nsteps= 600
burnin=100
pos = starting_point + 1e-2*np.random.randn(nwalkers, ndim)

sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(M, M_lim_a))
sampler.run_mcmc(pos, nsteps)

#plot chain
plt.plot(np.transpose(sampler.chain[:,:,0]))
plt.show()
sampler.chain
samples = sampler.chain[:, burnin:, :].reshape((-1, ndim))
fig = corner.corner(samples, labels=["Alpha", "Log(M_c)"], label_kwargs={"fontsize": 18},
                                     quantiles=[0.16, 0.5, 0.84], show_titles=True, title_kwargs={"fontsize": 18})

fig.show()

AP_data_table: ['APID', 'RegID', 'logAge-Best', 'logAge-P16', 'logAge-P84', 'logMass-Best', 'logMass-P16', 'logMass-P84']
emcee: Exception while calling your likelihood function:
  params: [-1.98203918  3.95078966]
  args: [array([3.98, 3.41, 3.16, ..., 3.11, 2.51, 2.57]), array([3.03342376, 3.03342376, 3.03342376, ..., 3.03342376, 3.03342376,
       2.86923172])]
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/Users/Tobin/Dropbox/m33_zoo_data/data_v1/Tobin_Work_flow/emcee/src/emcee/ensemble.py", line 545, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "<ipython-input-31-b6d329ab84ed>", line 60, in lnprob
    return lp + lnlike(theta, M, M_lim_a)
  File "<ipython-input-31-b6d329ab84ed>", line 48, in lnlike
    return np.sum((-lin_M/lin_M_c) + alpha*np.log(lin_M) + lnobs_like(M, M_lim_a) - lnZ(theta, lin_M, M_lim_a))
  File "<ipython-input-31-b6d329ab84ed>", line 39, in lnZ
    ans, err = quad(f, M_lim_a[i], np.inf)
  File "/Users/Tobin/opt/anaconda3/lib/python3.7/site-packages/scipy/integrate/quadpack.py", line 342, in quad
    points)
  File "/Users/Tobin/opt/anaconda3/lib/python3.7/site-packages/scipy/integrate/quadpack.py", line 455, in _quad
    return _quadpack._qagie(func,bound,infbounds,args,full_output,epsabs,epsrel,limit)
KeyboardInterrupt


KeyboardInterrupt: 